# Distribution of Molecular Configurations

## Understanding Thermodynamic Entropy Through the Central Limit Theorem

**Interactive Demonstration**: This notebook explores how the Central Limit Theorem explains the emergence of thermodynamic behavior in macroscopic systems.

### Overview

We model N ink molecules diffusing in water, where each molecule has a 50% probability of being in the left or right half of the container. This simple binomial system demonstrates how microscopic randomness leads to deterministic macroscopic behavior, with relative fluctuations scaling as **1/√N**.

### Key Concepts

- For small N: Distribution is noticeably spread out
- For large N: Distribution becomes extremely narrow relative to the mean
- For N ≈ 10²³ (macroscopic): Relative width ~ 10⁻¹² → essentially deterministic

---

## 1. Setup and Imports

In [ ]:
# Install required packages (Uncomment the next line and run this cell if packages are missing)
# %pip install numpy scipy plotly ipywidgets nbformat==4.2.0

In [ ]:
# import packages

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.special import gammaln, comb
from scipy.stats import binom, norm
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
import warnings
warnings.filterwarnings('ignore')

print("✓ All packages loaded successfully!")

In [ ]:
import plotly.io as pio
from IPython.display import display

# Set plotly renderer
try:
    import google.colab  # Only exists in Colab
    pio.renderers.default = "colab"
    from google.colab import output
    output.enable_custom_widget_manager()
except ImportError:
    pio.renderers.default = "notebook"  # Or "inline" or your preferred default

## 2. Mathematical Functions

### Core Probability Distributions

We implement both exact binomial and normal approximations:

**Exact Binomial PMF:**
$$P(k) = \binom{N}{k} \cdot p^k \cdot (1-p)^{N-k} = \binom{N}{k} \cdot \left(\frac{1}{2}\right)^N$$

**Normal Approximation (CLT):**
$$f(k) \approx \frac{1}{\sigma\sqrt{2\pi}} \exp\left(-\frac{(k-\mu)^2}{2\sigma^2}\right)$$

where $\mu = N/2$ and $\sigma = \sqrt{N/4}$

In [ ]:
def log_factorial(x):
    """
    Compute log(x!) using Stirling's approximation for large x.
    For small x, compute exact value.
    """
    if x <= 1:
        return 0
    if x < 20:
        # Exact for small values
        return np.sum(np.log(np.arange(2, x + 1)))
    # Stirling's approximation
    return x * np.log(x) - x + 0.5 * np.log(2 * np.pi * x)

def log_binomial(n, k):
    """
    Compute log(C(n,k)) to avoid overflow.
    """
    if k > n or k < 0:
        return -np.inf
    if k == 0 or k == n:
        return 0
    return log_factorial(n) - log_factorial(k) - log_factorial(n - k)

def exact_binomial_pmf(k, n, p=0.5):
    """
    Compute EXACT binomial PMF using logarithmic methods.
    P(k) = C(n,k) * p^k * (1-p)^(n-k)
    """
    log_prob = log_binomial(n, k) + k * np.log(p) + (n - k) * np.log(1 - p)
    return np.exp(log_prob)

def normal_pdf(x, mu, sigma):
    """
    Normal probability density function.
    """
    coefficient = 1 / (sigma * np.sqrt(2 * np.pi))
    exponent = -np.power(x - mu, 2) / (2 * np.power(sigma, 2))
    return coefficient * np.exp(exponent)

def compute_entropy(p, N):
    """
    Compute normalized entropy using KL divergence.
    S = -N * [p*log2(p) + (1-p)*log2(1-p)]
    Returns: S/N (normalized entropy)
    """
    if p <= 0 or p >= 1:
        return 0
    entropy = -N * (p * np.log2(p) + (1 - p) * np.log2(1 - p))
    return entropy / N

print("✓ Mathematical functions defined")

## 3. Data Generation Function

In [ ]:
def generate_distribution_data(log_N, visual_mode='smooth'):
    """
    Generate distribution data based on N and visualization mode.
    
    Parameters:
    -----------
    log_N : int
        Logarithm base 10 of number of molecules
    visual_mode : str
        'smooth', 'discrete', or 'both'
    
    Returns:
    --------
    dict with distribution data and statistics
    """
    N = int(10**log_N)
    mean = N / 2
    sd = np.sqrt(N / 4)
    
    # Thresholds for computation methods
    use_exact_binomial = N <= 1_000_000
    use_discrete = N <= 100_000
    
    k_values = []
    fractions = []
    probabilities = []
    entropies = []
    
    if use_discrete and visual_mode in ['discrete', 'both']:
        # Discrete sampling: integer k values
        start = max(0, int(np.ceil(mean - 6 * sd)))
        end = min(N, int(np.floor(mean + 6 * sd)))
        
        for k in range(start, end + 1):
            fraction = (k / N) * 100
            
            # Use exact binomial if N is small enough
            if use_exact_binomial:
                prob = exact_binomial_pmf(k, N)
            else:
                prob = normal_pdf(k, mean, sd)
            
            # Entropy
            p = k / N
            entropy = compute_entropy(p, N)
            
            k_values.append(k)
            fractions.append(fraction)
            probabilities.append(prob)
            entropies.append(entropy)
    else:
        # Continuous sampling for smooth curves
        num_points = 300
        start = max(0, mean - 6 * sd)
        end = min(N, mean + 6 * sd)
        k_continuous = np.linspace(start, end, num_points)
        
        for k in k_continuous:
            fraction = (k / N) * 100
            prob = normal_pdf(k, mean, sd)
            
            # Entropy
            p = k / N
            entropy = compute_entropy(p, N)
            
            k_values.append(k)
            fractions.append(fraction)
            probabilities.append(prob)
            entropies.append(entropy)
    
    # Calculate statistics
    relative_sd = (sd / mean * 100)
    sigma_range_1 = [(mean - sd) / N * 100, (mean + sd) / N * 100]
    sigma_range_3 = [(mean - 3*sd) / N * 100, (mean + 3*sd) / N * 100]
    
    return {
        'N': N,
        'mean': mean,
        'sd': sd,
        'k_values': k_values,
        'log_N': log_N, # Add log_N to the data dictionary
        'fractions': fractions,
        'probabilities': probabilities,
        'entropies': entropies,
        'relative_sd': relative_sd,
        'sigma_range_1': sigma_range_1,
        'sigma_range_3': sigma_range_3,
        'is_discrete': use_discrete and visual_mode in ['discrete', 'both'],
        'is_exact': use_exact_binomial and use_discrete and visual_mode in ['discrete', 'both']
    }

print("✓ Data generation function defined")

## 4. Visualization Function

In [ ]:
def create_distribution_plot(data, visual_mode='smooth'):
    """
    Create an interactive plotly visualization of the distribution.
    """
    # Determine if we need a second y-axis for entropy
    show_entropy = visual_mode == 'both'
    
    if show_entropy:
        fig = make_subplots(specs=[[{"secondary_y": True}]])
    else:
        fig = go.Figure()
    
    # Main probability plot
    if data['is_discrete'] and visual_mode == 'discrete':
        # Bar chart for discrete mode
        fig.add_trace(
            go.Bar(
                x=data['fractions'],
                y=data['probabilities'],
                name='Probability (PMF)',
                marker_color='rgba(99, 102, 241, 0.6)',
                hovertemplate='<b>%{x:.6f}% in left half</b><br>' +
                              'Probability: %{y:.4e}<br>' +
                              '<extra></extra>'
            ),
            secondary_y=False if show_entropy else None
        )
    else:
        # Line chart for smooth mode
        fig.add_trace(
            go.Scatter(
                x=data['fractions'],
                y=data['probabilities'],
                name='Probability Density',
                mode='lines',
                line=dict(color='rgb(99, 102, 241)', width=3),
                hovertemplate='<b>%{x:.6f}% in left half</b><br>' +
                'Probability Density: %{y:.4e}<br>' + 
                '<extra></extra>'
            ),
            secondary_y=False if show_entropy else None
        )
    
    # Add entropy curve if in 'both' mode
    if show_entropy:
        fig.add_trace(
            go.Scatter(
                x=data['fractions'],
                y=data['entropies'],
                name='Relative Entropy',
                mode='lines',
                line=dict(color='rgb(16, 185, 129)', width=2, dash='dash'),
                hovertemplate='<b>%{x:.6f}% in left half</b><br>' +
                              'Rel. Entropy: %{y:.6f}<br>' +
                              '<extra></extra>'
            ),
            secondary_y=True
        )
    
    # Add reference line at 50%
    fig.add_vline(
        x=50,
        line_dash="dash",
        line_color="red",
        annotation_text="Uniform (50%)",
        annotation_position="top"
    )
    
    # Update layout
    title_text = f"Probability Distribution for N = 10^{data['log_N']} = {data['N']:.2e} Molecules"
    if data['is_exact']:
        title_text += " <i>(Exact Binomial PMF)</i>"
    elif data['is_discrete']:
        title_text += " <i>(Normal Approximation)</i>"
    
    fig.update_layout(
        title=title_text,
        xaxis_title="Percentage of molecules in left half (%)",
        height=500,
        hovermode='closest',
        template='plotly_white',
        showlegend=True,
        legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
    )
    
    # Update y-axes
    if show_entropy:
        if data['is_discrete']:
            fig.update_yaxes(title_text="Probability", type="log", secondary_y=False)
        else:
            fig.update_yaxes(title_text="Probability Density", type="log", secondary_y=False)
        fig.update_yaxes(title_text="Relative Entropy", range=[0, 1], secondary_y=True)
    else:
        if data['is_discrete']:
            fig.update_yaxes(title_text="Probability", type="log")
        else:
            fig.update_yaxes(title_text="Probability Density", type="log")
    
    return fig

print("✓ Visualization function defined")

## 5. Statistics Display Function

In [ ]:
def display_statistics(data):
    """
    Display key statistics in a formatted manner.
    """
    stats_html = f"""
    <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 15px; margin: 20px 0;">
        <div style="background: #EFF6FF; padding: 15px; border-radius: 8px; border: 2px solid #BFDBFE;">
            <p style="color: #6B7280; font-size: 14px; margin: 0;">Peak (Most Probable)</p>
            <p style="color: #2563EB; font-size: 24px; font-weight: bold; margin: 5px 0;">50.000000%</p>
            <p style="color: #9CA3AF; font-size: 12px; margin: 0;">Uniform distribution</p>
        </div>
        <div style="background: #ECFDF5; padding: 15px; border-radius: 8px; border: 2px solid #A7F3D0;">
            <p style="color: #6B7280; font-size: 14px; margin: 0;">Relative Std Dev</p>
            <p style="color: #10B981; font-size: 20px; font-weight: bold; margin: 5px 0;">{data['relative_sd']:.8f}%</p>
            <p style="color: #9CA3AF; font-size: 12px; margin: 0;">σ/μ = 1/√N</p>
        </div>
        <div style="background: #F5F3FF; padding: 15px; border-radius: 8px; border: 2px solid #DDD6FE;">
            <p style="color: #6B7280; font-size: 14px; margin: 0;">±1σ Range</p>
            <p style="color: #8B5CF6; font-size: 16px; font-weight: bold; margin: 5px 0;">
                {data['sigma_range_1'][0]:.6f}%<br/>to {data['sigma_range_1'][1]:.6f}%
            </p>
            <p style="color: #9CA3AF; font-size: 12px; margin: 0;">68% probability</p>
        </div>
        <div style="background: #FFF7ED; padding: 15px; border-radius: 8px; border: 2px solid #FED7AA;">
            <p style="color: #6B7280; font-size: 14px; margin: 0;">±3σ Range</p>
            <p style="color: #F97316; font-size: 16px; font-weight: bold; margin: 5px 0;">
                {data['sigma_range_3'][0]:.8f}%<br/>to {data['sigma_range_3'][1]:.8f}%
            </p>
            <p style="color: #9CA3AF; font-size: 12px; margin: 0;">99.7% probability</p>
        </div>
    </div>
    """
    display(HTML(stats_html))

print("✓ Statistics display function defined")

## 6. Interactive Visualization

### Controls:
- **Number of Molecules**: Adjust from 10² to 10²³ (1 mole)
- **Visualization Mode**: Choose between smooth curve, discrete bars, or both with entropy

**Run the cell below to launch the interactive demo:**

In [ ]:
def update_visualization(log_N, visual_mode):
    """
    Main function to update visualization based on controls.
    """
    # Map dropdown values to internal mode names
    mode_map = {
        'Smooth (Normal PDF)': 'smooth',
        'Discrete (Binomial PMF)': 'discrete',
        'Both (with Entropy)': 'both'
    }
    mode = mode_map[visual_mode]
    
    # Generate data
    data = generate_distribution_data(log_N, mode)
    
    # Display info banner
    N = data['N']
    info_text = f"### Configuration: N = 10^{log_N} = {N:.2e} molecules"
    
    if N <= 100_000:
        if N <= 1_000_000:
            info_text += " | ✓ Exact binomial available"
        else:
            info_text += " | Using normal approximation"
    else:
        info_text += " | N too large, using smooth normal"
    
    display(Markdown(info_text))
    
    # Create and display plot
    fig = create_distribution_plot(data, mode)
    display(fig)
    
    # Display statistics
    display_statistics(data)
    
    # Display insights
    insights = f"""
    ### 🔍 Key Insights:
    
    - **Distribution Type**: {'Exact Binomial PMF (computed for N=' + f'{N:.0e}' + ')' if data['is_exact'] 
                              else 'Normal approximation (discrete bars)' if data['is_discrete'] 
                              else 'Normal PDF (continuous curve)'}
    - **Peak Position**: Exactly at 50% (uniform distribution) - maximum entropy state
    - **Width Scaling**: Standard deviation σ = √(N/4), so relative width σ/μ = 1/√N = {data['relative_sd']:.8f}%
    - **Macroscopic Limit**: For N = 10²³, width ≈ 10⁻¹² %, making uniform distribution the only observable state
    """
    display(Markdown(insights))

# Create interactive widgets
log_N_slider = widgets.IntSlider(
    value=12,
    min=2,
    max=23,
    step=1,
    description='log₁₀(N):',
    style={'description_width': 'initial'},
    continuous_update=False
)

mode_dropdown = widgets.Dropdown(
    options=['Smooth (Normal PDF)', 'Discrete (Binomial PMF)', 'Both (with Entropy)'],
    value='Smooth (Normal PDF)',
    description='Mode:',
    style={'description_width': 'initial'}
)

# Create interactive output
interactive_plot = widgets.interactive_output(
    update_visualization,
    {'log_N': log_N_slider, 'visual_mode': mode_dropdown}
)

# Display controls
controls = widgets.VBox([
    widgets.HTML("<h3>Interactive Controls</h3>"),
    log_N_slider,
    mode_dropdown
])

display(controls)
display(interactive_plot)

---

## 7. Mathematical Background

### Exact vs. Approximation

#### Exact Binomial Distribution (Discrete)

For N molecules, each independently choosing left (L) or right (R) with probability p = 0.5:

$$P(k) = \binom{N}{k} \left(\frac{1}{2}\right)^N$$

This is the **exact** probability mass function (PMF). For computational efficiency with large N, we use logarithmic methods:

$$\log P(k) = \log\binom{N}{k} - N \log 2$$

#### Normal Approximation (Central Limit Theorem)

For large N, the binomial distribution converges to a normal distribution:

$$P(k) \approx \frac{1}{\sigma\sqrt{2\pi}} \exp\left(-\frac{(k-\mu)^2}{2\sigma^2}\right)$$

where:
- $\mu = Np = N/2$ (mean)
- $\sigma = \sqrt{Np(1-p)} = \sqrt{N/4}$ (standard deviation)

### Key Result: Relative Width Scaling

The relative standard deviation (coefficient of variation):

$$\frac{\sigma}{\mu} = \frac{\sqrt{N/4}}{N/2} = \frac{1}{\sqrt{N}}$$

This **1/√N scaling** is crucial:
- For N = 10²: relative width ≈ 10%
- For N = 10⁶: relative width ≈ 0.1%
- For N = 10²³: relative width ≈ 10⁻¹² %

### Entropy Connection

The thermodynamic entropy for a macrostate with fraction p in the left half:

$$S = -k_B N \left[p \ln p + (1-p) \ln(1-p)\right]$$

This reaches its maximum at p = 0.5 (uniform distribution), which is exactly where the probability distribution peaks!

## 8. Exploration Exercises

Try these experiments with the interactive visualization above:

### Exercise 1: Small System Behavior
Set N = 10² and use **Discrete mode**:
- Observe the noticeable spread in the distribution
- Notice that deviations from 50% are quite probable
- Count how many distinct states are visible

### Exercise 2: Convergence to CLT
Gradually increase N from 10² → 10⁴ → 10⁶:
- Watch how the discrete bars merge into a smooth curve
- Observe the relative width shrinking
- Compare exact binomial vs. normal approximation

### Exercise 3: Macroscopic Limit
Set N = 10²³ (one mole):
- Note the incredibly narrow distribution (width ~ 10⁻¹² %)
- This explains why we never observe significant deviations from equilibrium
- The "deterministic" behavior of thermodynamics emerges from statistics

### Exercise 4: Entropy Analysis
Use **Both mode** to see entropy alongside probability:
- Maximum entropy occurs at exactly 50% (uniform distribution)
- This coincides with the probability peak
- Entropy drops rapidly for non-uniform distributions

---

## 9. Technical Implementation Notes

### Computational Considerations

1. **Threshold Selection**:
   - N ≤ 10⁶: Can compute exact binomial PMF using log-space arithmetic
   - N > 10⁶: Must use normal approximation to avoid numerical overflow
   - Discrete visualization practical only for N ≤ 10⁵

2. **Logarithmic Methods**:
   - Direct computation of C(N,k) causes overflow for large N
   - Using log(C(N,k)) = log(N!) - log(k!) - log((N-k)!) avoids overflow
   - Stirling's approximation for log(n!) ≈ n log(n) - n + 0.5 log(2πn)

3. **Sampling Strategy**:
   - Discrete mode: Sample integer k values (computationally intensive)
   - Smooth mode: Sample 300 points uniformly in the ±6σ range
   - Focus on ±6σ region (captures >99.9999% of probability)

---

## Summary and Conclusions

### What This Demo Shows

1. **Microscopic Randomness**: Each molecule independently chooses left or right
2. **Macroscopic Determinism**: For large N, only the uniform distribution is observable
3. **CLT Connection**: The 1/√N scaling explains the emergence of thermodynamic behavior
4. **Entropy Maximum**: The most probable state (50/50) is also the maximum entropy state

### Pedagogical Value

This demonstration bridges familiar probability concepts (CLT, binomial distribution) with thermodynamic principles:
- **No quantum mechanics needed**: Uses only undergraduate probability
- **Exact calculations possible**: Can verify CLT convergence numerically
- **Intuitive scaling**: The 1/√N law is easy to understand and calculate
- **Direct connection**: Links probability distributions to entropy

### Extensions

This framework can be extended to:
- Multi-compartment systems (multinomial distributions)
- Energy distributions (exponential/Boltzmann factors)
- Phase space volumes in statistical mechanics
- Fluctuation-dissipation theorems

---

## Citation

This notebook accompanies the research paper:

**"Understanding Thermodynamic Entropy Through the Central Limit Theorem: A Pedagogical Approach"**

*Submitted for publication, 2025*

---

### About

**Author**: [Moksh Jayanth]  
**Institution**: BMS College of Engineering, Bengaluru  
**Contact**: [mokshjayanth@gmail.com]  
**Repository**: [https://github.com/mokshjayanth]

---

*Notebook created: November 2025*